In [9]:
!ls

HTT  Untitled.ipynb  chromedriver-linux64  chromedriver-linux64.zip


In [13]:
!ls ./chromedriver-linux64/chromedriver-linux64/chromedriver

./chromedriver-linux64/chromedriver-linux64/chromedriver


In [ ]:
!wget https://dl.google.com/linux/direct/google-chrome-stable_current_amd64.deb
!sudo apt install ./google-chrome-stable_current_amd64.deb
!google-chrome --version

In [ ]:
!pip install lxml
!sudo apt install libxml2-dev libxslt1-dev python3-dev

In [21]:
!pip install pyarrow pyspark

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 434.1/434.1 MB 2.7 MB/s eta 0:00:0000:0100:02
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 203.0/203.0 KB 7.8 MB/s eta 0:00:00
done
  Created wheel for pyspark: filename=pyspark-4.0.0-py2.py3-none-any.whl size=434741258 sha256=3f7ada909c80119b70140de3167499df8ad9b16d18550f5505c425ef6b33e50e
  Stored in directory: /home/amon/.cache/pip/wheels/62/69/eb/eef3014e40bbcff88f1d6dd762baebf6bf5d0266ba57be1ef8
Successfully built pyspark


In [22]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
import pandas as pd
import time

# Setup
service = Service('./chromedriver-linux64/chromedriver-linux64/chromedriver') 
options = webdriver.ChromeOptions()
options.add_argument('--headless')
driver = webdriver.Chrome(service=service, options=options)

def scrape_cost_of_living_table(url):
    driver.get(url)
    time.sleep(3)  # Wait for JS to load table, adjust if needed

    # Find all tables (each section is a separate table)
    tables = driver.find_elements(By.TAG_NAME, 'table')
    dataframes = []

    for table in tables:
        html = table.get_attribute('outerHTML')
        df = pd.read_html(html)[0]
        dataframes.append(df)

    return dataframes

# URLs
cork_url = "https://salaryaftertax.com/cost-of-living-calculator/ireland/cork"
dublin_url = "https://salaryaftertax.com/cost-of-living-calculator/ireland/dublin"

cork_tables = scrape_cost_of_living_table(cork_url)
dublin_tables = scrape_cost_of_living_table(dublin_url)

driver.quit()

# Example: show first table from Cork
print(cork_tables[0])


      Results Monthly Weekly
0  Net Income  €3,217   €742
1    Expenses  €1,902   €439
2     Savings  €1,314   €303


In [2]:
cork_tables

[      Results Monthly Weekly
 0  Net Income  €3,217   €742
 1    Expenses  €1,902   €439
 2     Savings  €1,314   €303,
        Unnamed: 0   Yearly Monthly Weekly
 0      Net Income  €38,599  €3,217   €742
 1  Expenses (59%)  €22,828  €1,902   €439
 2    Rent (1-bed)  €15,192  €1,266   €292
 3       Utilities   €2,920    €243    €56
 4       Groceries   €3,900    €325    €75
 5       Transport     €816     €68    €16
 6   Savings (41%)  €15,771  €1,314   €303,
   Apartment Type (Cork City) Average Monthly Rent
 0                  3-Bedroom               €1,893
 1                  2-Bedroom               €1,532
 2                  1-Bedroom               €1,266
 3           Shared Apartment                 €631,
                             Expense Average Monthly Cost
 0  Utilities  (1-bedroom apartment)            1 ×  €150
 1                          Internet             1 ×  €55
 2                        TV Licence          1 ×  €13.33
 3                        Phone Plan          

In [11]:
import pickle

with open('cork_tables.pkl', 'wb') as f:
    pickle.dump(cork_tables, f)

with open('dublin_tables.pkl', 'wb') as f:
    pickle.dump(dublin_tables, f)

In [14]:
import pyarrow as pa
import pyarrow.parquet as pq
import os

os.makedirs("data/cork", exist_ok=True)
os.makedirs("data/dublin", exist_ok=True)

# Save each table as its own Parquet file
for i, df in enumerate(cork_tables):
    table = pa.Table.from_pandas(df)
    pq.write_table(table, f"data/cork/table_{i}.parquet")

for i, df in enumerate(dublin_tables):
    table = pa.Table.from_pandas(df)
    pq.write_table(table, f"data/dublin/table_{i}.parquet")

In [18]:
!ls data/cork

table_0.parquet  table_2.parquet  table_4.parquet  table_6.parquet
table_1.parquet  table_3.parquet  table_5.parquet


In [29]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("CostOfLiving").getOrCreate()

df_cork = spark.read.parquet("data/cork")

Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
25/06/10 16:43:06 WARN Utils: Your hostname, LAPTOP-J76M3340, resolves to a loopback address: 127.0.1.1; using 10.255.255.254 instead (on interface lo)
25/06/10 16:43:06 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/06/10 16:43:07 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
                                                                                

In [28]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
import time

service = Service('./chromedriver-linux64/chromedriver-linux64/chromedriver')

options = webdriver.ChromeOptions()
# DO NOT use headless if you're logging in manually
driver = webdriver.Chrome(service=service, options=options)

# Go to Facebook login
driver.get("https://www.facebook.com/login")

# Manually log in
input("Log in manually, then press ENTER here...")

# Now go to Marketplace for electric bikes near Mongolia
driver.get("https://www.facebook.com/marketplace/you/search/?query=electric%20bike")

# Wait for dynamic content to load
time.sleep(5)  # can bump this up

# Scroll down to load more items
for _ in range(5):  # or more if needed
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(3)

visible_text = driver.find_element("tag name", "body").text

with open("marketplace_visible_text.txt", "w", encoding="utf-8") as f:
    f.write(visible_text)

# Optional: screenshot
driver.save_screenshot("marketplace_screenshot.png")

driver.quit()


Log in manually, then press ENTER here... 


In [26]:
!ls

Untitled.ipynb		  cork_tables.pkl    marketplace_dump.html
chromedriver-linux64	  data		     marketplace_screenshot.png
chromedriver-linux64.zip  dublin_tables.pkl
